## Time-stepping-methods

In [ ]:
import sympy as sp
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
A,M = sp.symbols('A M', commutative=False)
x, k, v = sp.symbols('x k_n (k_{n}\lambda)')

In [ ]:
#gets the dimension of the space, the regularity, for which the basis functions will be used
#and as option nodal points (Warning: not tested if those were well chosen)
def bfg(q, reg = -1, xpoints = []):
    #case constant functions
    if(q == 1 and reg < 0):
        return [1]
    
    #test if the dimension of the space is high enough
    if(q-reg - 2 < 0):
        print('error: the dimension of the trial space is too low for the regularity')
        return []
    
    
    basis = []
    #construct nodal points if not given
    if(len(xpoints) == 0):
        if(reg > 0):
            if(q-2*reg > 1): #if additional nodal points have to be added
                xpoints = [sp.Rational(i,(q-1-2*reg)) for i in range(q-2*reg)]
            else:
                xpoints = [0,1]
        else:
            xpoints = [sp.Rational(i,(q-1)) for i in range(q)] #reg<1        
    
    ypoints = [0 for i in range(q)]
    
    #Construct matrix for matrix-vector-equation to fullfill the properties at the nodal points
    #case reg > 0
    if(reg > 0):
        xmatrix = [[0 for b in range(q)] for a in range(reg+1)] 
        xmatrix += [[1] + [xpoints[a+1] for b in range(q-1)] for a in range(q-2*reg-2)] 
        xmatrix[0][0] = 1
        
        for i in range(reg+1):
            xmatrix[i][i] = factorial(i)
            
        if(q-2*reg > 1): #if additional nodal points have been added
            xmatrix += [[0 for b in range(q)] for a in range(reg+1)]
            for i in range(reg+1):
                for j in range(i,q):
                    xmatrix[-(1+i)][j] = factorial(j)//factorial(j-i)
            for i in range(q-1):
                for j in range(len(xpoints)-2):
                    xmatrix[reg+1+j][i+1] *= xmatrix[reg+1+j][i]

        else:
            xmatrix += [[0 for b in range(q)] for a in range(q-(reg+1))]
            for i in range(q-reg-1):
                for j in range(i,q):
                    xmatrix[-(1+i)][j] = factorial(j)//factorial(j-i)
        
    #case reg < 1
    else:
        xmatrix = [[1] + [xpoints[a] for b in range(q-1)] for a in range(q)]
        for i in range(q-1):
            for j in range(q):
                xmatrix[j][i+1] *= xmatrix[j][i]
                

            
    xmatrix = sp.Matrix(xmatrix)
    
    #loop over all equations
    for l in range(q):
        #solve matrix-vector-equation and get the factors
        ypoints[l] = 1

        yvector = sp.Matrix(ypoints)
        factor = xmatrix**(-1)*yvector
        
        #assemble basis function
        expr = 0        
        for i in range(q):
            exponent = 1
            for j in range(i):
                exponent *= x 
            expr += factor[i]*exponent
        basis.append(expr)
        ypoints[l] = 0
        
    return basis

In [ ]:
#to differentiate polynomials (makes it possible to differentiate constants)
def diff(poly,x,i):
    if(type(poly) == type(1)):
        return 0
    return sp.sympify(poly.diff(x,i))

#pluggs numbers in polynomials (makes it possible to aply it on constants)
def sub(poly,y,n):
    if(type(poly) == type(1)):
        return poly
    return poly.subs(y,n)

In [ ]:
#gets a list of basis vectors and plots it
def plotbasefnc(basis,title = '',save=False):
    points = 200
    l = np.arange(0, (points+1)/points, 1/points)
    for j in range(len(basis)):
        y = []
        string = '$\hat{\phi}_'+str(j) + ' = ' + sp.latex(eval(str(basis[j]))) + '$'
        for i in range(points+1):
            y.append(sub(basis[j],x,l[i]))
        plt.plot(l,y,'-', label=string)
    plt.grid()
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    #plt.title(title)
    savetitle = title.replace(' ','_')
    if save:
        plt.savefig(savetitle+'.jpg', bbox_inches='tight', dpi=150)
    plt.show()

In [ ]:
#function gets list of basisfunction with rising degree, s.t. they are suitable as testfunctions,
#as option the regularity (dG = -1 default), 
#the number of collacation points qp (default is 0) and a list of suitable (WARNING: not tested) test functions
#returns the 'prefactor matrix' for U^n and the 'prefactor matrix' for U^{n-1}
def makematrices(basis, reg = -1, qp = 0, test = []):
    #too many collacation points
    if((reg > 0) and (len(basis)-reg-2-qp < 0)):
        print("error: too many collocation points. The dim of the test space would be too small")
        return
    
    #defining testfunctions
    if(len(test) == 0):
        test = bfg(len(basis)-(reg+1+qp))

    #Jumpterm in DG case
    if(reg == -1): 
        D = [[sub(i,x,0)*sub(j,x,0) for j in basis] for i in test]
        D1 = [[sub(i,x,0)*sub(j,x,1) for j in basis] for i in test]

    #Regularity equations
    else:
        D = [[0 for i in basis] for j in test] 
        D += [[0 for i in basis] for j in range(qp)] 
        D += [[sub(j,x,0) for j in basis]]
        D += [[sub(diff(j,x,i),x,0) for j in basis] for i in range(1,reg+1)]
        
        D1 = [[0 for i in basis] for j in test]
        D1 += [[0 for i in basis] for j in range(qp)]
        D1 += [[sub(j,x,1) for j in basis]] 
        D1 += [[sub(diff(j,x,i),x,1) for j in basis] for i in range(1,reg+1)]


    #construct matrices with the testfunctions
    A1 = [[sp.Integral((j)*i,(x,0,1)).doit() for j in basis] for i in test]
    A1 += [[sub(j,x,1-sp.Rational(i,qp)) for j in basis] for i in range(qp)]
    A1 += [[0 for j in range(len(basis))] for i in range(reg+1)]
    
    T = [[sp.Integral(diff(j,x,1)*(i),(x,0,1)).doit() for j in basis] for i in test]
    T += [[sub(diff(j,x,1),x,1-sp.Rational(i,qp)) for j in basis] for i in range(qp)]
    T += [[0 for j in range(len(basis))] for i in range(reg+1)]
    
    
    return (M*sp.Matrix(D) + M*sp.Matrix(T) + k*A*sp.Matrix(A1)), M*sp.Matrix(D1)

In [ ]:
eigen = []

#finds the max eigenvalue from the global eigenvalue list 'eigen' evaluated at a
def  maxev(a):
    mev = 0
    for i in eigen:
        q = abs(i.subs(v, a).evalf())
        mev = max(q,mev)
    return mev

#gets the method matrices and options for the picture of the stability region and plots it
def plotsr(P, Q,size = 20, ran = 10e0, title = ''):
    global eigen
    
    #find eigenvalues of the method matrix dependend on the eigenvalue lambda of A
    P = P.subs(M,1)
    P = P.subs(k,1)
    P = P.subs(A,-v)
    Q = Q.subs(M,1)
    P = P**(-1)*Q
    eigen = list(P.eigenvals())
    print('Eigenvalues are:')
    print(eigen)

    #define evaluation grid
    xlist = np.linspace(-ran,ran,size)
    ylist = np.linspace(-ran,ran,size)
    X, Y = np.meshgrid(xlist,ylist)
    Z = X + 1J * Y
    
    #plot
    fig,ax = plt.subplots(1,1)
    cp = ax.contourf(X, Y, np.vectorize(maxev)(Z), levels = [-1e99,1], colors = 'pink')
    ax.set_xlabel('Re')
    ax.set_ylabel('Im')
    plt.grid()
    if(title != ''):
        plt.savefig(title+'.jpg', bbox_inches='tight', dpi=150)
    plt.show()

In [ ]:
plotbasefnc(bfg(3,1))

Y,Z = makematrices(bfg(4,1),1,1)
plotsr(Y,Z,50,10e0)

## Convergence analysis

In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from xfem import *
from math import pi
from ngsolve.meshes import *
ngsglobals.msg_level = 1
import matplotlib.pyplot as plt

In [ ]:
#gets order in space and time and number of space cells as well as time step size
#returns the l2 and linfinity error for dG
def spacetimeDG(k_s, k_t, delta_h, delta_t):
    mesh = Make1DMesh(delta_h)
    
    V = H1(mesh, order=k_s, dirichlet=".*")
    tfe = ScalarTimeFE(k_t)
    st_fes = tfe * V
    
    tnew = 0
    tend = 1.0
    told = Parameter(0)
    t = told + delta_t * tref
    t = t.MakeVariable()

    u_exact = sin(pi * t) * sin(pi * x)

    coeff_f = u_exact.Diff(t) - u_exact.Diff(x).Diff(x)
    coeff_f = coeff_f.Compile()
    
    gfu = GridFunction(st_fes)
    u_last = CreateTimeRestrictedGF(gfu, 1)
    u, v = st_fes.TnT()

    dxt = delta_t * dxtref(mesh, time_order=2*k_t+2)
    dxold = dmesh(mesh, tref=0)
    dxnew = dmesh(mesh, tref=1)
    def dt(u):
        return 1.0 / delta_t * dtref(u)
    
    a = BilinearForm(st_fes, symmetric=False)
    a += grad(u) * grad(v) * dxt
    a += u * v * dxold
    a += dt(u) * v * dxt
    a.Assemble()
    ainv = a.mat.Inverse(st_fes.FreeDofs())

    f = LinearForm(st_fes)
    f += coeff_f * v * dxt
    f += u_last * v * dxold
    
    u_last.Set(fix_tref(u_exact, 0))
    Draw(u_last, mesh, "u")
    
    l2errors = []
    tolds = []

    while tend - told.Get() > delta_t / 2:
        f.Assemble()
        gfu.vec.data = ainv * f.vec
        RestrictGFInTime(spacetime_gf=gfu, reference_time=1.0, space_gf=u_last)
        l2error = Integrate((u_exact - gfu)**2 * dxnew, mesh)
        Redraw()
        told.Set(told.Get() + delta_t)
        l2errors.append(l2error)
        tolds.append(told.Get())
        #print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error))

    lmaxnorm = max([sqrt(i) for i in l2errors])
    l2norm = sqrt(sum([i*delta_t for i in l2errors]))

    return lmaxnorm,l2norm

In [ ]:
#gets order in space and time and number of space cells as well as time step size
#returns the l2 and linfinity error for dG
def spacetimeCG(k_s, k_t, delta_h, delta_t):
    assert(k_t>=1)
    #mesh
    mesh = Make1DMesh(delta_h)
    
    # Spatial FESpace for solution
    fes = H1(mesh, order=k_s, dirichlet=".*")
    # Time finite elements (nodal!)
    tfe_i = ScalarTimeFE(k_t, skip_first_node=True)  # interior shapes
    tfe_e = ScalarTimeFE(k_t, only_first_node=True)  # exterior shapes (init. val.)
    tfe_t = ScalarTimeFE(k_t - 1)                    # test shapes
    # Space-time finite element space
    st_fes_i, st_fes_e, st_fes_t = [tfe * fes for tfe in [tfe_i, tfe_e, tfe_t]]
    
    
    #para
    tnew = 0
    tend = 1.0
    told = Parameter(0)
    t = told + delta_t * tref
    t = t.MakeVariable()
    
    u_exact = sin(pi * t) * sin(pi * x)

    coeff_f = u_exact.Diff(t) - u_exact.Diff(x).Diff(x)
    coeff_f = coeff_f.Compile()
    
    gfu_i = GridFunction(st_fes_i)
    gfu_e = GridFunction(st_fes_e)

    u_last = CreateTimeRestrictedGF(gfu_e, 0)

    u_i = st_fes_i.TrialFunction()
    u_e = st_fes_e.TrialFunction()
    v_t = st_fes_t.TestFunction()
    
    dxt = delta_t * dxtref(mesh, time_order=2*k_t+2)
    dxold = dmesh(mesh, tref=0)
    dxnew = dmesh(mesh, tref=1)
    def dt(u):
        return 1.0 / delta_t * dtref(u)
    
    a_i = BilinearForm(trialspace=st_fes_i,testspace=st_fes_t, symmetric=False)
    a_i += grad(u_i) * grad(v_t) * dxt
    a_i += dt(u_i) * v_t * dxt
    a_i.Assemble()
    a_iinv = a_i.mat.Inverse(st_fes_i.FreeDofs())

    
    f = LinearForm(st_fes_t)
    f += coeff_f * v_t * dxt 
    f += -grad(gfu_e) * grad(v_t) * dxt
    f += -dt(gfu_e) * v_t * dxt
    
    u_last.Set(fix_tref(u_exact, 0))
    Draw(u_last, mesh, "u")
    
    l2errors = []
    tolds = []
    
    while tend - told.Get() > delta_t / 2:
        gfu_e.vec.data = u_last.vec
        f.Assemble()
        gfu_i.vec.data = a_iinv * f.vec
        RestrictGFInTime(spacetime_gf=gfu_i, reference_time=1.0, space_gf=u_last)
        l2error = Integrate((u_exact - gfu_i)**2 * dxnew, mesh)
        Redraw()
        told.Set(told.Get() + delta_t)
        l2errors.append(l2error)
        tolds.append(told.Get())
        #print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error))
        
    lmaxnorm = max([sqrt(i) for i in l2errors])
    l2norm = sqrt(sum([i*delta_t for i in l2errors])) #korrekt?
    
    return lmaxnorm,l2norm

In [ ]:
#gets order in space and time and number of space cells as well as time step size
#returns the l2 and linfinity error for G_1(4)
def spacetimeGCC(k_s, k_t, delta_h, delta_t):
    assert(k_t==3)
    #mesh
    mesh = Make1DMesh(delta_h)
    
    # Spatial FESpace for solution
    fes = H1(mesh, order=k_s, dirichlet=".*")
    
    # time finite elements (nodal!)
    tfe_i = GCC3FE(skip_first_nodes=True)  # interior shapes
    tfe_e = GCC3FE(only_first_nodes=True)  # exterior shapes (init. val.)
    tfe_t = ScalarTimeFE(0)                    # test shapes
    # space-time finite element space
    st_fes_i, st_fes_e, st_fes_t = [tfe * fes for tfe in [tfe_i, tfe_e, tfe_t]]

    fes_t = FESpace([st_fes_t,fes])

    tnew = 0
    tend = 1.0
    told = Parameter(0)
    t = told + delta_t * tref
    t = t.MakeVariable()
    
    u_exact = sin(pi * t) * sin(pi * x)

    coeff_f = u_exact.Diff(t) - u_exact.Diff(x).Diff(x)
    coeff_f = coeff_f.Compile()
    
    u_i = st_fes_i.TrialFunction()
    u_e = st_fes_e.TrialFunction()
    v_t, w_t  = fes_t.TestFunction()
    
    gfu_i = GridFunction(st_fes_i)
    gfu_e = GridFunction(st_fes_e)

    u_last = CreateTimeRestrictedGF(gfu_e)

    dxt = delta_t * dxtref(mesh, time_order=2*3+2)
    dxold = dmesh(mesh, tref=0)
    dxnew = dmesh(mesh, tref=1)
    
    def dt(u): return 1 / delta_t * dtref(u)

    a_i = BilinearForm(trialspace=st_fes_i, testspace=fes_t, symmetric=False)
    a_i += grad(u_i) * grad(v_t) * dxt
    a_i += dt(u_i) * v_t * dxt
    a_i += grad(u_i) * grad(w_t) * dxnew
    a_i += dt(u_i) * w_t * dxnew

    a_i.Assemble()
    
    a_iinv = a_i.mat.Inverse(st_fes_i.FreeDofs())
    
    a_e = BilinearForm(trialspace=st_fes_e, testspace=fes_t, symmetric=False)
    a_e += grad(u_e) * grad(v_t) * dxt
    a_e += dt(u_e) * v_t * dxt
    a_e += grad(u_e) * grad(w_t) * dxnew
    a_e += dt(u_e) * w_t * dxnew
    a_e.Assemble()
    
    f = LinearForm(fes_t)
    f += coeff_f * v_t * dxt
    f += coeff_f * w_t * dxnew
    
    u_last.Set(delta_t*fix_tref(u_exact.Diff(t), 0))
    gfu_e.vec[fes.ndof:2*fes.ndof] = u_last.vec[:]
    u_last.Set(fix_tref(u_exact, 0))
    
    gfu_e.vec[0:fes.ndof] = u_last.vec
    Draw(u_last, mesh, "u")
    
    l2errors = []
    tolds = []
    
    while tend - told.Get() > delta_t / 2:
        f.Assemble()
        f.vec.data -= a_e.mat * gfu_e.vec
        gfu_i.vec.data = a_iinv * f.vec
        gfu_e.vec.data = gfu_i.vec
        RestrictGFInTime(spacetime_gf=gfu_i, reference_time=1.0, space_gf=u_last)
        l2error = Integrate((u_exact -gfu_i)**2 * dxnew, mesh)
        Redraw()
        told.Set(told.Get() + delta_t)
        l2errors.append(l2error)
        tolds.append(told.Get())
        
        gfu_e.vec[0:fes.ndof] = gfu_i.vec[0:fes.ndof]
        gfu_e.vec[fes.ndof:2*fes.ndof] = gfu_i.vec[fes.ndof:2*fes.ndof]
        #print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error))
        
    lmaxnorm = max([sqrt(i) for i in l2errors])
    l2norm = sqrt(sum([i*delta_t for i in l2errors])) #korrekt?

    return lmaxnorm,l2norm

In [ ]:
#gets order in space and time and prints latex convergence table
def convergenceinftynorm(k_s, k_t):
    timesteps = [1/2**i for i in range(9)]
    h = 1000
    l2errors = []
    lmaxerrors = []
    for tau in timesteps:
        l2er, lmaxer = spacetimeCG(k_s, k_t, h, tau) #change here the method
        l2errors.append(l2er)
        lmaxerrors.append(lmaxer)


    eocl2 = [0]
    eoclmax = [0]

    for i in range(1,len(l2errors)):
        eocl2.append(log(l2errors[i-1]/l2errors[i])/log(timesteps[i-1]/timesteps[i]))
        eoclmax.append(log(lmaxerrors[i-1]/lmaxerrors[i])/log(timesteps[i-1]/timesteps[i]))
        
    #make latex tabel
    print("\\begin{tabular}{lrrrr}")
    print("\\toprule")
    print("j & $\\lVert u-U \\rVert_{\infty,j}$ & $EOC_j^{\infty}$ & $\\lVert u-U \\rVert_{2,j}$ & $EOC_j^2$ \\\\")
    print("\\midrule")
    for i in range(len(timesteps)):
        linestr = str(i) + "&" + str(lmaxerrors[i]) + "&" + str(round(eoclmax[i],3))
        linestr = linestr + "&" + str(l2errors[i]) + "&" + str(round(eocl2[i],2)) + "\\\\"
        print(linestr)
    print("\\bottomrule")
    print("\end{tabular}\\newline")

In [ ]:
convergenceinftynorm(4,4)